In [20]:
import sklearn
from sklearn.decomposition import PCA
#use pandas to load csv file
import pandas as pandas
#use numpy to calculate stuff
import numpy as np

#load data and merge both tables to one, ignore_index to reindex
redwinedata = pandas.read_csv('data/winequality-red.csv', sep =';')
whitewinedata = pandas.read_csv('data/winequality-white.csv', sep =';')

#simplified data for testing
#redwinedata = pandas.read_csv('data/red_onlysugar.csv', sep =';')
#whitewinedata = pandas.read_csv('data/white_onlysugar.csv', sep =';')

concat_data = redwinedata.append(whitewinedata, ignore_index=True)
# drop the quality label and normalize the data
concat_data = concat_data.drop('quality', axis=1)
winearray = concat_data.values
winearray_norm = sklearn.preprocessing.scale(winearray)

In [34]:
#https://stackoverflow.com/questions/23294616/how-to-use-scikit-learn-pca-for-features-reduction-and-know-which-features-are-d
pca = PCA(n_components=3)
pca.fit(winearray_norm)
#print(pca.explained_variance_ratio_)  
print(pca.components_[0])


[-0.2387989  -0.3807575   0.15238844  0.34591993 -0.29011259  0.43091401
  0.48741806 -0.04493664 -0.21868644 -0.29413517 -0.10643712]


pca.components_[0, i] to the rest of pca.components_[0]